<a href="https://colab.research.google.com/github/maclandrol/cours-ia-med/blob/master/04_Classification_Texte_Medical_Francais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04. Classification de Texte Médical Français

**Enseignant:** Emmanuel Noutahi, PhD

---

**Objectif:** Apprendre à classifier des textes médicaux en français en utilisant des approches de prompting et d'apprentissage automatique.

**Dataset:** FrenchMedMCQA - Questions à choix multiples de pharmacie

**Approches:**
- Prompting avec ChatGPT
- Classification automatique avec CamemBERT

## Installation et Configuration

In [ ]:
!pip install "datasets<4.0.0" transformers torch pandas scikit-learn matplotlib seaborn -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositif: {device}")

## 1. Accès aux Données

In [ ]:
# Chargement du dataset FrenchMedMCQA
dataset = load_dataset("qanastek/FrenchMedMCQA", trust_remote_code=True)
print(f"Dataset chargé: {len(dataset['train'])} exemples")

## 2. Transformation en DataFrame

In [ ]:
# Conversion en DataFrame
df = pd.DataFrame(dataset['train'])
print(f"DataFrame: {df.shape}")
print(f"Colonnes: {list(df.columns)}")

## 3. Analyse dataset


In [ ]:
# Statistiques de base
print(f"Nombre de questions: {len(df)}")
print(f"Longueur moyenne des questions: {df['question'].str.len().mean():.0f} caractères")

# Distribution des réponses correctes
answer_counts = df['correct_answers'].apply(len).value_counts().sort_index()
print("\nNombre de bonnes réponses par question:")
for nb, count in answer_counts.items():
    print(f"  {nb} réponse(s): {count} questions")

In [ ]:
# Exemple de question
exemple = df.iloc[0]
print("Exemple de question:")
print(f"Q: {exemple['question']}")
print(f"Options: {exemple['options']}")
print(f"Réponses correctes: {exemple['correct_answers']}")

## 4. Prompting avec ChatGPT

### Le Rôle du Prompting

Le prompting consiste à formuler des instructions claires pour guider un modèle de langue vers la réponse souhaitée. Pour les questions médicales, un bon prompt doit:
- Définir le rôle d'expert médical
- Préciser le format de réponse attendu
- Fournir le contexte nécessaire

### Prompt Simple

In [ ]:
# Génération d'un prompt simple
def generer_prompt_simple(question, options):
    options_text = "\n".join([f"{chr(65+i)}. {option}" for i, option in enumerate(options)])
    
    prompt = f"""Tu es un expert médical français. Pour cette question d'examen de pharmacie, indique toutes les réponses correctes.

Question: {question}

Options:
{options_text}

Réponses correctes (lettres uniquement):"""
    
    return prompt

# Test avec notre exemple
prompt_simple = generer_prompt_simple(exemple['question'], exemple['options'])
print("PROMPT SIMPLE À COPIER DANS CHATGPT:")
print("=" * 50)
print(prompt_simple)
print("=" * 50)

### Prompt Few-Shot (Apprentissage en Contexte)

L'apprentissage en contexte (in-context learning) consiste à fournir quelques exemples dans le prompt pour guider le modèle. Cette technique améliore souvent les performances sans nécessiter d'entraînement.

In [ ]:
def generer_prompt_few_shot(question, options):
    options_text = "\n".join([f"{chr(65+i)}. {option}" for i, option in enumerate(options)])
    
    prompt = f"""Tu es un expert médical français. Voici des exemples de questions d'examen de pharmacie avec leurs réponses correctes:

Exemple 1:
Question: Quels sont les effets indésirables de l'aspirine?
Options:
A. Troubles digestifs
B. Amélioration de l'humeur
C. Risque hémorragique
D. Augmentation de l'appétit
Réponses correctes: A, C

Exemple 2:
Question: Le paracétamol est contre-indiqué en cas de:
Options:
A. Insuffisance hépatique sévère
B. Hypertension artérielle
C. Diabète
D. Grossesse
Réponses correctes: A

Maintenant, réponds à cette nouvelle question:

Question: {question}

Options:
{options_text}

Réponses correctes (lettres uniquement):"""
    
    return prompt

# Test few-shot
prompt_few_shot = generer_prompt_few_shot(exemple['question'], exemple['options'])
print("PROMPT FEW-SHOT À COPIER DANS CHATGPT:")
print("=" * 50)
print(prompt_few_shot)
print("=" * 50)

## 5. Classification Automatique

### Préparation des Données

In [ ]:
# Préparation pour la classification binaire (simplifiée)
# Prédire si chaque option est correcte ou non

def preparer_donnees_classification(df, max_samples=1000):
    df_sample = df.head(max_samples).copy()
    
    texts = []
    labels = []
    
    for _, row in df_sample.iterrows():
        question = row['question']
        options = row['options']
        correct_answers = row['correct_answers']
        
        # Pour chaque option, créer un exemple
        for i, option in enumerate(options):
            text = f"{question} [SEP] {option}"
            texts.append(text)
            
            # Label: 1 si cette option est correcte, 0 sinon
            label = 1 if chr(65+i) in correct_answers else 0
            labels.append(label)
    
    return texts, labels

texts, labels = preparer_donnees_classification(df, max_samples=500)
print(f"Données préparées: {len(texts)} exemples")
print(f"Labels positifs: {sum(labels)} ({sum(labels)/len(labels)*100:.1f}%)")

In [ ]:
# Division train/test
texts_train, texts_test, labels_train, labels_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Train: {len(texts_train)} exemples")
print(f"Test: {len(texts_test)} exemples")

### Entraînement du Modèle

In [ ]:
# Chargement du modèle CamemBERT
model_name = "camembert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

print(f"Modèle chargé: {model_name}")

In [ ]:
# Tokenisation
def tokenize_function(texts):
    return tokenizer(texts, truncation=True, padding=True, max_length=256, return_tensors="pt")

train_encodings = tokenize_function(texts_train)
test_encodings = tokenize_function(texts_test)

print("Tokenisation terminée")

In [ ]:
# Dataset PyTorch
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = MedicalDataset(train_encodings, labels_train)
test_dataset = MedicalDataset(test_encodings, labels_test)

In [ ]:
# Configuration d'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Fonction de métriques
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Entraînement
print("Début de l'entraînement...")
trainer.train()
print("Entraînement terminé")

## 6. Évaluation et Performance

In [ ]:
# Évaluation sur le jeu de test
results = trainer.evaluate()
print("Résultats d'évaluation:")
for metric, value in results.items():
    if metric.startswith('eval_'):
        metric_name = metric.replace('eval_', '')
        print(f"{metric_name}: {value:.3f}")

In [ ]:
# Prédictions détaillées
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = labels_test

print("Rapport de classification:")
print(classification_report(y_true, y_pred, target_names=['Incorrect', 'Correct']))

In [ ]:
# Test sur un nouvel exemple
def predire_reponse(question, option, model, tokenizer):
    text = f"{question} [SEP] {option}"
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        prediction = torch.argmax(probabilities, dim=-1)
    
    return prediction.item(), probabilities[0][1].item()

# Test sur l'exemple
test_question = exemple['question']
test_options = exemple['options']
correct_answers = exemple['correct_answers']

print(f"Question: {test_question[:100]}...")
print(f"Réponses attendues: {correct_answers}")
print("\nPrédictions du modèle:")

for i, option in enumerate(test_options):
    pred, confidence = predire_reponse(test_question, option, model, tokenizer)
    status = "CORRECT" if pred == 1 else "INCORRECT"
    expected = "✓" if chr(65+i) in correct_answers else "✗"
    print(f"{chr(65+i)}. {status} (conf: {confidence:.3f}) [{expected}]")

## Résumé

### Approches Testées

1. **Prompting Simple**: Instructions directes pour ChatGPT
2. **Few-Shot Learning**: Utilisation d'exemples dans le prompt pour guider le modèle
3. **Classification Automatique**: Entraînement d'un modèle CamemBERT spécialisé

### Applications Pratiques

- **Prompting**: Idéal pour des cas d'usage ponctuels ou des domaines très spécialisés
- **Classification**: Recommandée pour des volumes importants et des besoins de rapidité
- **Hybride**: Combiner prompting pour la validation et classification pour le traitement en masse

Le prochain notebook explorera l'analyse d'images médicales avec TorchXRayVision.